# Recurrent Neural Networks

RNNs son un tipo especial de arquitecturas de RN construida en base a bloques constructores que se conectan de forma cíclica. Es decir, reciben como entrada la salida anterior del bloque constructor. Este tipo de arquitectura trata de emular el funcionamiento de la memoria, resultando especialmente útil para aplicaciones donde la entrada se proporciona en secuencia en vez de toda al mismo tiempo. En concreto, el procesamiento del lenguajes natural, el reconocimiento de voz, o el procesamiento de señal, son aplicaciones en las que es útil un mecanismo de este tipo.

En Pytorch tenemos un soporte nativo para la implementación de este tipo de redes. Por ejemplo, en el paquete torch.nn, tenemos un conjunto de bloques constructores ya implementados, útils para la construcción de este tipo de redes. En concreto:

- RNN, RNNBase, RNNCell
- LSTM, LSTMCell
- GRU, GRUCell

A continuación vamos a ver un ejemplo completo de implementación de una RNN en Pytorch.

In [5]:
import torch
from torch import nn

# Creación sintética de datos de entrada: una secuencia de números del 0 al 99
input_data = torch.arange(0, 100, dtype=torch.float32)
# Creación de datos de salida: una secuencia de números del 1 al 100, con el último elemento desplazado al primero
target_data = torch.roll(input_data, shifts=-1)
# El último elemento de la secuencia de salida es 100, que no tiene correspondencia en la secuencia de entrada
target_data[-1] = 100.0

# Cambiar la forma de los datos de entrada (batch, seq_len, input_size) (batch size es 1)
input_data = torch.reshape(input_data, (1, -1, 1))
target_data = torch.reshape(target_data, (1, -1, 1))

class SimpleRNN(nn.Module):
    def __init__(self):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(
            input_size=1,
            hidden_size=32,     # número de unidades en la capa oculta
            num_layers=1,       # número de capas de la RNN
            batch_first=True,   # usamos el format (batch, seq, feature) para la entrada
        )
        self.out = nn.Linear(32, 1)

    def forward(self, x):
        r_out, _ = self.rnn(x, None)
        out = self.out(r_out)
        return out

# Creación y entrenaiento del modelo
model = SimpleRNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):
    model.train()
    output = model(input_data)
    loss = criterion(output, target_data)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

# Prueba del modelo
model.eval()
test_input = torch.tensor([50.0]).reshape(1, 1, 1)
print("Given input 50.0, predicted output:", model(test_input).item())


Epoch: 0, Loss: 3332.966796875
Epoch: 100, Loss: 2937.9755859375
Epoch: 200, Loss: 2651.806884765625
Epoch: 300, Loss: 2395.025146484375
Epoch: 400, Loss: 2163.60888671875
Epoch: 500, Loss: 1954.666259765625
Epoch: 600, Loss: 1765.7923583984375
Epoch: 700, Loss: 1594.957763671875
Epoch: 800, Loss: 1440.370361328125
Epoch: 900, Loss: 1300.4566650390625
Given input 50.0, predicted output: 29.38585090637207
